In [1]:
# ── Full Reload, Retrain & Back-test (fixed) ────────────────────────
import importlib, pandas as pd
import algo.features as feat_mod
import algo.model    as model_mod
importlib.reload(feat_mod); importlib.reload(model_mod)

from algo.features   import add_indicators, FEATURES
from algo.model      import load_or_train, predict_last, LOOKBACK
from algo.broker     import KiteWrapper
from algo.config     import load_config
from algo.backtester import backtest

# 1️⃣ Pull 200 days of 3-min history and build features
cfg    = load_config()
broker = KiteWrapper(cfg)
hist_all = broker.history(days=200, interval="3minute",
                          tradingsymbol="HDFCBANK")
df_all   = add_indicators(hist_all, debug=True).ffill()

# 2️⃣ Split by *calendar days* instead of row count
n_test_days = 20                          # last 20 trading days ≈ 1500 bars
cutoff      = df_all.index[-1] - pd.Timedelta(days=n_test_days)

df_tr = df_all[df_all.index <  cutoff]    # ~180 days
df_te = df_all[df_all.index >= cutoff]    #  last 20 days  (≳ LOOKBACK rows)

print(f"Training rows: {len(df_tr):,}")
print(f"Testing  rows: {len(df_te):,}  (need ≥ {LOOKBACK})")

# 3️⃣ (Re)train the model
model = load_or_train(df_tr, retrain=False)

# 4️⃣ Inspect the pipeline
print("\n▶️ Pipeline steps:", model.named_steps)
gb = model.named_steps["gb"]
print("▶️ Final estimator:", gb)
print("▶️ n_features_in_:", gb.n_features_in_)
print(f"▶️ Expected      : {len(FEATURES)} × {LOOKBACK} = "
      f"{len(FEATURES)*LOOKBACK}\n")

# 5️⃣ Run back-test
trades, metrics = backtest(
    df_te,
    model=model,
    capital=200_000,
    contract_size=1,
    sl_pct=0.0010,
    tp_pct=0.0025,
    trail_pct=0.0020,
    hold_max=10,
    upper=0.85,
    lower=0.20,
    predict_fn=predict_last,
    debug=True,
)

# 6️⃣ Review results
print("Back-test metrics:", metrics)
print("\nFirst few trades:\n", trades.head())


[broker] loaded from /Users/sreejit/PycharmProjects/zerodha-bot/algo/broker.py
[KiteWrapper] initialized: symbol=RELIANCE on exch=NSE
[history] start: days=200, interval=3minute, symbol=HDFCBANK
[history] range UTC-naive: 2024-12-17 08:06:06.302470 → 2025-07-05 08:06:06.302470
[history] token=341249
[history] got 8750 bars, cursor→2025-03-26 15:30:00
[history] got 8375 bars, cursor→2025-07-04 15:30:00
[history] empty data for 2025-07-04 15:30:00->2025-07-05 08:06:06.302470, breaking loop
[history] complete 17125 bars 2024-12-17 09:15:00 → 2025-07-04 15:27:00
ret1 head: [0.0, -0.0015866612881538789, 2.6935301405961454e-05, 0.00035014948689626735, -0.0009423801830910072]
ema_3 head: [1859.25, 1857.775, 1857.0625, 1857.03125, 1856.140625]
ema_8 head: [1859.25, 1858.5944444444444, 1858.0956790123457, 1857.85219478738, 1857.2739292790734]
vwap head: [1858.2333333333333, 1858.1054515067092, 1857.913397812432, 1857.8600306476915, 1857.7290373499936]
vol_spike head: [0, 0, 0, 0, 0]
Training ro

In [3]:
trades

,exit_ts,side,entry_price,exit_price,exit_reason,fees,qty,pnl,equity
entry_ts,,,,,,,,,
2025-06-20 15:18:00,2025-06-23 09:15:00,SELL,1967.3,1961.39810,TP,68.576541,101,527.515359,200527.515359
2025-06-23 09:24:00,2025-06-23 09:54:00,SELL,1951.5,1949.50000,TIME,68.630083,102,135.369917,200662.885276
2025-06-23 09:57:00,2025-06-23 10:27:00,SELL,1949.4,1949.00000,TIME,68.613354,102,-27.813354,200635.071923
2025-06-23 10:33:00,2025-06-23 11:03:00,SELL,1950.5,1948.30000,TIME,68.618305,102,155.781695,200790.853618
2025-06-24 09:18:00,2025-06-24 09:27:00,SELL,1973.5,1967.57950,TP,68.643910,101,529.326590,201320.180208
2025-06-24 10:30:00,2025-06-24 10:48:00,SELL,1971.2,1965.28640,TP,68.830987,102,534.356213,201854.536421
2025-06-24 10:54:00,2025-06-24 11:03:00,SELL,1966.6,1969.54990,SL,68.815577,102,-369.705377,201484.831044
2025-06-24 11:06:00,2025-06-24 11:27:00,SELL,1971.8,1974.75770,SL,68.872732,102,-370.558132,201114.272912
2025-06-24 11:30:00,2025-06-24 12:00:00,SELL,1973.7,1976.66055,SL,68.680933,101,-367.696483,200746.576429
